# IA - Criptomoedas

In [1]:
from pyknow import *
import requests, json

request = requests.get("https://api.coinmarketcap.com/v2/ticker/")
json = json.loads(str(request.content.decode("utf-8")))["data"]

In [2]:
json

{'1': {'circulating_supply': 17295700.0,
  'id': 1,
  'last_updated': 1538254343,
  'max_supply': 21000000.0,
  'name': 'Bitcoin',
  'quotes': {'USD': {'market_cap': 114186211574.0,
    'percent_change_1h': 0.17,
    'percent_change_24h': -0.76,
    'percent_change_7d': -1.48,
    'price': 6602.00001008,
    'volume_24h': 4348515052.23499}},
  'rank': 1,
  'symbol': 'BTC',
  'total_supply': 17295700.0,
  'website_slug': 'bitcoin'},
 '1027': {'circulating_supply': 102265871.0,
  'id': 1027,
  'last_updated': 1538254356,
  'max_supply': None,
  'name': 'Ethereum',
  'quotes': {'USD': {'market_cap': 23647340753.0,
    'percent_change_1h': -0.63,
    'percent_change_24h': 4.62,
    'percent_change_7d': -3.66,
    'price': 231.233945271,
    'volume_24h': 2155614057.50092}},
  'rank': 2,
  'symbol': 'ETH',
  'total_supply': 102265871.0,
  'website_slug': 'ethereum'},
 '1042': {'circulating_supply': 37114165509.0,
  'id': 1042,
  'last_updated': 1538254405,
  'max_supply': None,
  'name': 'S

In [44]:
# Representa o model de uma moeda
class Coin(Fact):
    name = Field(str, mandatory=True)
    price = Field(float, mandatory=True)
    percentChange1h = Field(float, mandatory=True)
    percentChange24h = Field(float, mandatory=True)
    percentChange7d = Field(float, mandatory=True)
    
# Representa um contexto de escolhas
class Context(Fact):
    pass

# Representa a escolha de uma moeda
class CoinName(Fact):
    pass


In [73]:
investValue = 450

class CoinCoinEngine(KnowledgeEngine):

    '''
        DefFacts#0
        
        Declara todas as moedas que serão consumidas nas regras
    '''
    @DefFacts(order=0)
    def declareCoins(self):
        yield Coin(name="BitCoin", price=450.0, percentChange1h=0.0, percentChange24h=-3.0, percentChange7d=-5.0)
        yield Coin(name="IOTA", price=450.0, percentChange1h=0.5, percentChange24h=-0.5, percentChange7d=-5.0)
        
    '''
        DefFacts#1
        
        Entrada do nome da moeda e inicialização das regras
    '''
    @DefFacts(order=1)
    def startup(self):
        global investValue, intendedValue
        coin = input("Put the coin name: ")
        self.declare(CoinName(coinName=coin))
        yield Context(state="verificaMoeda")
        
    '''
        Rule#1
        
        Verifica se existe a moeda entrada pelo usuário
    '''
    @Rule(AS.context << Context(state="verificaMoeda"),
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          NOT(Coin(name=MATCH.coinName)))
    def verificaMoedaInexistente(self):
        print("[Rule#1] Moeda inexistente")
        
    '''
        Rule#2
        
        Se existir a moeda altera o contexto para seguir para a próxima regra        
    '''
    @Rule(AS.context << Context(state="verificaMoeda"),
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          Coin(name=MATCH.coinName))
    def verificaMoedaExistente(self, context):
        print("[Rule#2] Moeda válida")
        self.modify(context, state="verificaValorInvestido")
        
    '''
        Rule#3
        
        Verifica se o valor investido é menor que o valor unitário da moeda
    '''
    @Rule(AS.context << Context(state="verificaValorInvestido"), 
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          AS.coin << Coin(name=MATCH.coinName, price=MATCH.price & GT(investValue)))
    def verificaValorInvestidoErrado(self, context, coin, contextCoinName):
        print("[Rule#3] Você não possui dinheiro suficiente")
    
    '''
        Rule#4
        
        Se o valor investido for permitido avança para a próxima regra    
    '''
    @Rule(AS.context << Context(state="verificaValorInvestido"), 
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          AS.coin << Coin(name=MATCH.coinName, price=MATCH.price & LE(investValue)))
    def verificaValorInvestido(self, context, coin, contextCoinName):
        print("[Rule#4] Você pode investir")
        self.modify(context, state="decisao1")
    
    '''
        Rule#5
        
        Verifica se a tendencia da moeda é desvalorizar (1h)
    '''
    @Rule(
          AS.context << Context(state="decisao1"), 
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          AS.coin << Coin(name=MATCH.coinName, percentChange1h=MATCH.percentChange1h & LT(0))
    )
    def decisao1Negativa(self, context, coin, contextCoinName):
        print("[Result] A moeda está desvalorizando na última hora, não é seguro investir")
    
    '''
        Rule#6

        Verifica se a tendencia da moeda é valorizar (1h)
    '''
    @Rule(
          AS.context << Context(state="decisao1"), 
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          AS.coin << Coin(name=MATCH.coinName, percentChange1h=MATCH.percentChange1h & GE(0))
    )
    def decisao1Positiva(self, context, coin, contextCoinName):
        print("[Rule#6] Na última hora a moeda valorizou")
        self.modify(context, state="decisao2")
        
    '''
        Rule#7

        Verifica se não houve grande desvalorização nas últimas 24h
    '''
    @Rule(
          AS.context << Context(state="decisao2"), 
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          AS.coin << Coin(name=MATCH.coinName, percentChange24h=MATCH.percentChange24h & GE(-3))
    )
    def decisao2Positiva(self, context, coin, contextCoinName):
        print("[Rule#7] Nas últimas 24 horas a moeda não desvalorizou expressivamente")
        self.modify(context, state="decisao3")
        
    '''
        Rule#8

        Verifica se houve grande desvalorização nas últimas 24h
    '''
    @Rule(
          AS.context << Context(state="decisao2"), 
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          AS.coin << Coin(name=MATCH.coinName, percentChange24h=MATCH.percentChange24h & LT(-3))
    )
    def decisao2Negativa(self, context, coin, contextCoinName):
        print("[Result] A moeda está desvalorizando nas últimas 24 horas, não é seguro investir")
        
        
    '''
        Rule#9

        Verifica se não houve grande desvalorização nos últimos 7 dias
    '''
    @Rule(
          AS.context << Context(state="decisao3"), 
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          AS.coin << Coin(name=MATCH.coinName, percentChange7d=MATCH.percentChange7d & GE(-10))
    )
    def decisao3Positiva(self, context, coin, contextCoinName):
        print("[Rule#9] Nos últimos 7 dias a moeda não desvalorizou expressivamente")
        print("[Result] É seguro investir")
        
    '''
        Rule#10

        Verifica se houve grande desvalorização nos últimos 7 dias
    '''
    @Rule(
          AS.context << Context(state="decisao3"), 
          AS.contextCoinName << CoinName(coinName=MATCH.coinName),
          AS.coin << Coin(name=MATCH.coinName, percentChange7d=MATCH.percentChange7d & LT(-10))
    )
    def decisao3Negativa(self, context, coin, contextCoinName):
        print("[Result] A moeda está desvalorizando nos últimos 7 dias, não é seguro investir")


In [75]:
engine = CoinCoinEngine()
engine.reset()
engine.run()

Put the coin name: 2


[Rule#2] Moeda válida
[Rule#4] Você pode investir
[Rule#6] Na última hora a moeda valorizou
[Rule#7] Nas últimas 24 horas a moeda não desvalorizou expressivamente
[Result] A moeda está desvalorizando nos últimos 7 dias, não é seguro investir
